# **PRÀCTICA 1: Identificació d'idioma**

In [1]:
import re

In [4]:
import os

texts = {}
directory = os.fsencode('./original_langId')
for file in os.listdir(directory):
	filename = os.fsdecode(file)
	directoryname = os.fsdecode(directory)
	with open(directoryname + '/' + filename, 'r', encoding="utf-8") as file:
		data = file.read()
		texts[filename] = data
	

In [5]:
def preprocess_text(text: str) -> str:
    new_text = ""
    sents = text.split('\n')
    for sent in sents:
        sent.strip()
        sent = sent.lower()
        sent = re.sub(r'\d', '', sent)
        sent = re.sub(r'\s+', ' ', sent)
        new_text += "  " + sent
    return new_text

In [ ]:
for file, text in texts.items():
    f = open("./new_langID/" + file, "w", encoding="utf-8")
    f.write(preprocess_text(text))
    f.close()